# Align Murcko Scaffolds

In [1]:
%load_ext autoreload
%autoreload 2

### Libraries

In [2]:
import open3d as o3d
import numpy as np
import seaborn as sns
import pandas as pd

from numpy.random import default_rng

import re, os
from io import StringIO

from tqdm.auto import trange

import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools

import ipywidgets as widgets

import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-07 11:31:40,716 - __init__ - Enabling RDKit 2021.03.3 jupyter extensions


In [3]:
from utils import show_molecule_idx
from utils import AlignShow, translate_and_rotate

In [4]:
from rdkit.Chem.Scaffolds import MurckoScaffold as MS

In [5]:
path = "ligands/CDK2"
files = [os.path.join(path, f) for f in os.listdir(path) if os.path.splitext(f)[-1] == ".pcd" and os.path.splitext(f)[0][-4:] == "tran"]

# Order ligands
# This should make the three chemical series pop-up in the PCD fit
names = {
    "4ek4_B_1CK": "CS1",
    "4ek5_B_03K": "CS3",
    "4fkg_B_4CK": "CS4",
    "4fki_B_09K": "CS9",
    "4fkj_B_11K": "CS11",
    "3sw4_B_18K": "CS18",
    "3sw7_B_19K": "CS19",
    "4fko_B_20K": "CS20",
    "4fkp_B_LS5": "CS241",
    "4fkq_B_42K": "CS242",
    "4fkr_B_45K": "CS245",
    "4fks_B_46K": "CS246",
    "4fkt_B_48K": "CS248",
    "4fku_D_60K": "CS260",
    "4fkv_B_61K": "CS261",
    "4fkw_B_62K": "CS262",
}

files.sort(key=lambda f: int(names[os.path.splitext(os.path.basename(f))[0].replace("_tran", "")].replace("CS", "")))

print(files)

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(f)
    pcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf"))
    mol = next(s)
    mols.append(mol)

['ligands/CDK2/4ek4_B_1CK_tran.pcd', 'ligands/CDK2/4ek5_B_03K_tran.pcd', 'ligands/CDK2/4fkg_B_4CK_tran.pcd', 'ligands/CDK2/4fki_B_09K_tran.pcd', 'ligands/CDK2/4fkj_B_11K_tran.pcd', 'ligands/CDK2/3sw4_B_18K_tran.pcd', 'ligands/CDK2/3sw7_B_19K_tran.pcd', 'ligands/CDK2/4fko_B_20K_tran.pcd', 'ligands/CDK2/4fkp_B_LS5_tran.pcd', 'ligands/CDK2/4fkq_B_42K_tran.pcd', 'ligands/CDK2/4fkr_B_45K_tran.pcd', 'ligands/CDK2/4fks_B_46K_tran.pcd', 'ligands/CDK2/4fkt_B_48K_tran.pcd', 'ligands/CDK2/4fku_D_60K_tran.pcd', 'ligands/CDK2/4fkv_B_61K_tran.pcd', 'ligands/CDK2/4fkw_B_62K_tran.pcd']


In [6]:
_ = widgets.interact(lambda index: show_molecule_idx(index, mols), index=widgets.IntSlider(min=0, max=len(mols)-1, step=1, value=1))

interactive(children=(IntSlider(value=1, description='index', max=15), Output()), _dom_classes=('widget-intera…

In [7]:
msMols = [MS.GetScaffoldForMol(mol) for mol in mols]

In [8]:
_ = widgets.interact(lambda index: show_molecule_idx(index, msMols), index=widgets.IntSlider(min=0, max=len(mols)-1, step=1, value=1))

interactive(children=(IntSlider(value=1, description='index', max=15), Output()), _dom_classes=('widget-intera…

Write scaffolds to file:

In [9]:
for idx, mol in enumerate(msMols):    
    # Randomly translate and rotate Murcko scaffolds
    translate_and_rotate(mol)

    with Chem.SDWriter(os.path.join(path, f"murcko_{idx}.sdf")) as w:
        w.write(mol, confId=0)

In [10]:
n_mols_minus_one = len(mols) - 1

In [11]:
%%bash -s "$path" "$n_mols_minus_one"

# Python variable path passed to bash
# Can be accessed with $1

# Python variable n_frags passed to bash
# Can be accessed with $2

# Unfortunately molgrid does not work well with pybel
# molgrid_to_pcd needs to run within a Singularity container
# (RDKit and molgrid compiled from source!)
# See https://github.com/gnina/libmolgrid/issues/62

for SCAFFOLD in $(seq 0 $2)
do
    singularity run --nv --app python ../development/densitymatch.sif \
        ../molgrid_to_pcd.py ${PWD}/${1}/murcko_${SCAFFOLD}.sdf -o ${PWD}/${1}/murcko_${SCAFFOLD}.pcd \
            --ligmap ${PWD}/../files/ligmap
done

In [12]:
!ls ligands/CDK2 | grep "murcko.*\.pcd"

murcko_0.pcd
murcko_10.pcd
murcko_11.pcd
murcko_12.pcd
murcko_13.pcd
murcko_14.pcd
murcko_15.pcd
murcko_1.pcd
murcko_2.pcd
murcko_3.pcd
murcko_4.pcd
murcko_5.pcd
murcko_6.pcd
murcko_7.pcd
murcko_8.pcd
murcko_9.pcd


In [13]:
mkpcds = []
mkmols = []

files = [os.path.join(path, f"murcko_{i}.pcd") for i in range(len(mols))]

print(files)

for f in files:
    pcd = o3d.io.read_point_cloud(f)
    mkpcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf"))
    mol = next(s)
    mkmols.append(mol)

print(mkpcds)

['ligands/CDK2/murcko_0.pcd', 'ligands/CDK2/murcko_1.pcd', 'ligands/CDK2/murcko_2.pcd', 'ligands/CDK2/murcko_3.pcd', 'ligands/CDK2/murcko_4.pcd', 'ligands/CDK2/murcko_5.pcd', 'ligands/CDK2/murcko_6.pcd', 'ligands/CDK2/murcko_7.pcd', 'ligands/CDK2/murcko_8.pcd', 'ligands/CDK2/murcko_9.pcd', 'ligands/CDK2/murcko_10.pcd', 'ligands/CDK2/murcko_11.pcd', 'ligands/CDK2/murcko_12.pcd', 'ligands/CDK2/murcko_13.pcd', 'ligands/CDK2/murcko_14.pcd', 'ligands/CDK2/murcko_15.pcd']
[PointCloud with 425 points., PointCloud with 490 points., PointCloud with 509 points., PointCloud with 572 points., PointCloud with 858 points., PointCloud with 602 points., PointCloud with 606 points., PointCloud with 597 points., PointCloud with 572 points., PointCloud with 633 points., PointCloud with 934 points., PointCloud with 729 points., PointCloud with 540 points., PointCloud with 541 points., PointCloud with 893 points., PointCloud with 567 points.]


In [14]:
molid = 8

# Add reference molecule to fragments
allmols = [mols[molid]] + mkmols
allpcds = [pcds[molid]] + mkpcds

In [15]:
als = AlignShow(allmols, allpcds)

Look at the best alignment between molecule `molid` (which has been appended at position `0`) any all Murcko's scaffolds.

In [16]:
s, (i,j) = als.best_with(0) # molid has been appended in position 0

In [17]:
s

0.7482598607888631

In [18]:
(i, j)

(0, 11)

In [19]:
als.scores

{(0, 0): 1.0,
 (0, 1): 0.3990719257540603,
 (0, 2): 0.45243619489559167,
 (0, 3): 0.4617169373549884,
 (0, 4): 0.5220417633410673,
 (0, 5): 0.5498839907192575,
 (0, 6): 0.46983758700696054,
 (0, 7): 0.41531322505800466,
 (0, 8): 0.4965197215777262,
 (0, 9): 0.4396751740139211,
 (0, 10): 0.5104408352668214,
 (0, 11): 0.7482598607888631,
 (0, 12): 0.345707656612529,
 (0, 13): 0.48607888631090485,
 (0, 14): 0.6299303944315545,
 (0, 15): 0.3839907192575406,
 (0, 16): 0.6276102088167054,
 (1, 0): 0.8635294117647059,
 (1, 1): 1.0,
 (1, 2): 0.7811764705882352,
 (1, 3): 0.8211764705882353,
 (1, 4): 0.7411764705882353,
 (1, 5): 0.76,
 (1, 6): 0.6564705882352941,
 (1, 7): 0.6423529411764706,
 (1, 8): 0.8023529411764706,
 (1, 9): 0.851764705882353,
 (1, 10): 0.6282352941176471,
 (1, 11): 0.6941176470588235,
 (1, 12): 0.7576470588235295,
 (1, 13): 0.8305882352941176,
 (1, 14): 0.7717647058823529,
 (1, 15): 0.6423529411764706,
 (1, 16): 0.8164705882352942,
 (2, 0): 0.7673469387755102,
 (2, 1): 0.83

With this dataset (where ligands come from the same chemical series), the best aligned scaffold is not necessarily `molid` because other scaffold can be similar but larger. The best scaffold is clearly well aligned and has a reasonable score:

In [20]:
_ = widgets.interact(lambda index: als.show(0, index), index=widgets.IntSlider(min=0, max=len(mols)-1, step=1, value=1))

interactive(children=(IntSlider(value=1, description='index', max=15), Output()), _dom_classes=('widget-intera…

In [21]:
als.save(i, j)
als.save(i, i)
als.save(j, j)